In [81]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Conv2D, MaxPooling2D, 
                                     Flatten, Dropout, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

In [82]:
# Load the dataset
data = pd.read_csv('fer2013.csv')

# Extract features and labels
pixels = data['pixels'].tolist()
emotions = data['emotion'].values

In [83]:
# Convert pixels to numpy arrays
X = np.array([np.fromstring(pixel_sequence, dtype='float32', sep=' ')
              for pixel_sequence in pixels])

In [84]:
# Normalize pixel values
X /= 255.0

In [85]:
# Reshape to (num_samples, 48, 48, 1)
X = X.reshape(-1, 48, 48, 1)

# Convert labels to categorical
y = to_categorical(emotions, num_classes=7)


In [86]:
from sklearn.utils import shuffle

# Shuffle the data
X, y = shuffle(X, y, random_state=42)

In [87]:
# Split into training and testing sets
num_samples = X.shape[0]
num_train_samples = int(0.8 * num_samples)  # 80% for training

In [88]:
X_train = X[:num_train_samples]
Y_train = y[:num_train_samples]
X_test = X[num_train_samples:]
Y_test = y[num_train_samples:]

In [89]:
# Convert grayscale images to RGB
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)


In [90]:
# 2. Data Augmentation

datagen = ImageDataGenerator(
    rotation_range=45,
    zoom_range=[0.5, 1.5],
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode='reflect'
)
datagen.fit(X_train_rgb)

In [91]:
# Custom data generator that only augments images
def data_generator(datagen, X, y, batch_size):
    genX = datagen.flow(X, batch_size=batch_size, shuffle=True, seed=42)
    while True:
        x_batch = genX.next()
        idx = (genX.batch_index - 1) * genX.batch_size
        if idx < 0:
            idx = len(X) - genX.batch_size
        y_batch = y[idx:idx + genX.batch_size]
        yield x_batch, y_batch

In [92]:
# 3. Building the Model with Transfer Learning

from tensorflow.keras.applications import ResNet50

def create_transfer_model(input_shape=(48, 48, 3), num_classes=7):
    # Load the ResNet50 model without the top layers
    base_model = ResNet50(weights='imagenet', include_top=False,
                          input_shape=input_shape)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [93]:
# Create the model
model = create_transfer_model()

In [94]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [95]:
callbacks = [
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        verbose=1,
        min_delta=1e-4
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        verbose=1,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        'best_transfer_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]


In [96]:
# Train the model without specifying steps_per_epoch
batch_size = 64
epochs = 50

train_generator = datagen.flow(X_train_rgb, Y_train, batch_size=batch_size, shuffle=True, seed=42)

history = model.fit(
    train_generator,
    validation_data=(X_test_rgb, Y_test),
    epochs=epochs,
    callbacks=callbacks,
    verbose=2
)


Epoch 1/50


c:\Users\Oana\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.25146, saving model to best_transfer_model.keras
449/449 - 161s - 358ms/step - accuracy: 0.1878 - loss: 2.2707 - val_accuracy: 0.2515 - val_loss: 2.0793 - learning_rate: 0.0010
Epoch 2/50

Epoch 2: val_accuracy improved from 0.25146 to 0.25300, saving model to best_transfer_model.keras
449/449 - 159s - 355ms/step - accuracy: 0.2218 - loss: 1.9018 - val_accuracy: 0.2530 - val_loss: 1.8072 - learning_rate: 0.0010
Epoch 3/50

Epoch 3: val_accuracy improved from 0.25300 to 0.25439, saving model to best_transfer_model.keras
449/449 - 157s - 350ms/step - accuracy: 0.2339 - loss: 1.8432 - val_accuracy: 0.2544 - val_loss: 1.8098 - learning_rate: 0.0010
Epoch 4/50


KeyboardInterrupt: 